In [ ]:
import json
import os

import agama
import h5py
import matplotlib.pyplot as plt
import numpy as np
from gc_utils import iteration_name, main_prog_halt, snapshot_name  # type: ignore
# from matplotlib.animation import PillowWriter

In [55]:
sim = "m12i"

sim_dir = "/Users/z5114326/Documents/simulations/"
data_dir = "/Users/z5114326/Documents/GitHub/gc_statistics/data/"
pot_dir = "/Users/z5114326/Documents/GitHub/gc_kinematics/data/potentials/"

fire_dir = sim_dir + sim + "/" + sim + "_res7100/"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

agama.setUnits(mass=1, length=1, velocity=1)

In [60]:
kin_dict = {}

it = 0
it_id = iteration_name(it)
source_dat = proc_data[it_id]["source"]

# group_id = 8580896

group_ids = np.unique(np.array(source_dat["group_id"]))
group_ids = group_ids[group_ids >= 0]

kin_dict[it_id] = {}

i = 1
for group_id in group_ids:
    print(str(i) + "/" + str(len(group_ids)))
    i += 1

    source_mask = (np.array(source_dat["group_id"]) == group_id) & (np.array(source_dat["analyse_flag"]) == 1)
    snap_acc = np.min(source_dat["snap_acc"][source_mask])

    key_lst = []
    for key in proc_data[it_id]["snapshots"].keys():
        # Check if group_id is in the group_id list for this snapshot
        if group_id in proc_data[it_id]["snapshots"][key]["group_id"]:
            # Check the conditions for snap_acc and key
            if (int(key[4:]) >= snap_acc) & (int(key[4:]) >= 46):
                key_lst.append(key)

    group_key = str(group_id)
    kin_dict[it_id][group_key] = {}
    for key in key_lst:
        snap_dict = proc_data[it_id]["snapshots"][key]
        snap_mask = np.array(snap_dict["group_id"]) == group_id

        snap = int(key[4:])
        pot_file = pot_dir + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)
        pot = agama.Potential(pot_file)
        af = agama.ActionFinder(pot, interp=False)

        et = np.array(snap_dict["et"][snap_mask])
        lz = np.array(snap_dict["lz"][snap_mask])

        if len(et) == 1:
            # pot.Rcirc it didn't like handling arrays of 1
            et = et[0]

        r_circs = pot.Rcirc(E=et)
        xyz = np.column_stack((r_circs, r_circs * 0, r_circs * 0))
        v_circs = np.sqrt(-r_circs * pot.force(xyz)[:, 0])
        vel = np.column_stack((v_circs * 0, v_circs, v_circs * 0))
        init_conds = np.concatenate((xyz, vel), axis=1)
        lz_circ = af(init_conds)[:, 2]

        E_0 = pot.potential((0, 0, 0))

        lz_norm = lz / np.array(lz_circ)
        et_norm = et / np.abs(E_0)

        kin_dict[it_id][group_key][key] = {}

        kin_dict[it_id][group_key][key]["lz"] = lz.tolist()
        kin_dict[it_id][group_key][key]["et"] = et.tolist()

        kin_dict[it_id][group_key][key]["lz_norm"] = lz_norm.tolist()
        kin_dict[it_id][group_key][key]["et_norm"] = et_norm.tolist()


save_dir = data_dir + "kin_dicts/" + sim + "/"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_file = save_dir + it_id + "_kin_dict.json"

with open(save_file, "w") as json_file:
    json.dump(kin_dict, json_file, indent=4)

1/33
2/33
3/33
4/33
5/33
6/33
7/33
8/33
9/33
10/33
11/33
12/33
13/33
14/33
15/33
16/33
17/33
18/33
19/33
20/33
21/33
22/33
23/33
24/33
25/33
26/33
27/33
28/33
29/33
30/33
31/33
32/33
33/33


In [61]:
proc_data.close()